## Project: Vehicle Detection and Tracking

### Import Packages
features-> train classifiers -> sliding window for searching

In [1]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import glob
import time
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
from sklearn.model_selection import train_test_split


### Import Images

In [3]:
#cars= glob.glob('./vehicles_smallset/**/*.jpeg')
#notcars = glob.glob('./non-vehicles_smallset/**/*.jpeg')

cars= glob.glob('./vehicles/**/*.png')
notcars = glob.glob('./non-vehicles/**/*.png')
print(len(cars))
print(len(notcars))
simage = mpimg.imread(cars[0])
print(np.max(simage))

8792
8968
1.0


### Color and Gradient Features

In [4]:

# Define a function to compute binned color features  
def bin_spatial(img, size=(32, 32)):
    # Use cv2.resize().ravel() to create the feature vector
    features = cv2.resize(img, size).ravel() 
    # Return the feature vector
    return features

# Define a function to compute color histogram features  
def color_hist(img, nbins=32, bins_range=(0, 256)):
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features

# Define a function to return HOG features and visualization
def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), block_norm= 'L2-Hys',
                                  transform_sqrt=True, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), block_norm= 'L2-Hys',
                       transform_sqrt=True, 
                       visualise=vis, feature_vector=feature_vec)
        return features



In [9]:
# Define a function to extract features from a list of images
# Have this function call bin_spatial() and color_hist()
def extract_features(imgs, cspace='RGB', spatial_size=(32, 32),
                        hist_bins=32, hist_range=(0, 256),orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        # Read in each one by one
        image = mpimg.imread(file)
        # apply color conversion if other than 'RGB'
        if cspace != 'RGB':
            if cspace == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif cspace == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif cspace == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif cspace == 'YCrCb':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
            elif cspace == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
        else: feature_image = np.copy(image)      
        # Apply bin_spatial() to get spatial color features
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        # Apply color_hist() also with a color space option now
        hist_features = color_hist(feature_image, nbins=hist_bins, bins_range=hist_range)
        # Call get_hog_features() with vis=False, feature_vec=True
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=False, feature_vec=True))
            hog_features = np.ravel(hog_features)        
        else:
            hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                        pix_per_cell, cell_per_block, vis=False, feature_vec=True)

        
        # Append the new feature vector to the features list
        #hog_features=[]
        features.append(np.concatenate((spatial_features, hist_features,hog_features)))
    # Return list of feature vectors
    return features


In [14]:
# performs under different binning scenarios
spatial = 16
histbin = 32
colorspace = 'YCrCb' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 9
pix_per_cell = 8
cell_per_block = 2
hog_channel = 0 # Can be 0, 1, 2, or "ALL"
hist_range = (0,1)
t1_f=time.time()

car_features = extract_features(cars, cspace=colorspace, spatial_size=(spatial, spatial),
                        hist_bins=histbin, hist_range=hist_range,orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
                        hog_channel=hog_channel)
notcar_features = extract_features(notcars, cspace=colorspace, spatial_size=(spatial, spatial),
                        hist_bins=histbin, hist_range=hist_range,orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
                        hog_channel=hog_channel)
t2_f=time.time()
print(t2_f-t1_f)

# Create an array stack of feature vectors
X = np.vstack((car_features, notcar_features)).astype(np.float64)

# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))

# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=rand_state)
    
# Fit a per-column scaler only on the training data
X_scaler = StandardScaler().fit(X_train)
# Apply the scaler to X_train and X_test
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

print('Using spatial binning of:',spatial,
    'and', histbin,'histogram bins')
print('Feature vector length:', len(X_train[0]))
# Use a linear SVC 
svc = LinearSVC()
# Check the training time for the SVC
t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
# Check the prediction time for a single sample
t=time.time()
n_predict = 10
print('My SVC predicts: ', svc.predict(X_test[0:n_predict]))
print('For these',n_predict, 'labels: ', y_test[0:n_predict])
t2 = time.time()
print(round(t2-t, 5), 'Seconds to predict', n_predict,'labels with SVC')

83.48264813423157
Using spatial binning of: 16 and 32 histogram bins
Feature vector length: 2628
17.53 Seconds to train SVC...
Test Accuracy of SVC =  0.9854
My SVC predicts:  [ 0.  0.  1.  0.  0.  0.  1.  1.  0.  1.]
For these 10 labels:  [ 0.  0.  1.  0.  0.  0.  1.  1.  0.  1.]
0.00276 Seconds to predict 10 labels with SVC


In [13]:
# # performs under different binning scenarios
# spatial = 16
# histbin = 32
# colorspace = 'RGB' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
# orient = 9
# pix_per_cell = 8
# cell_per_block = 2
# #hog_channel = 0 # Can be 0, 1, 2, or "ALL"
# hist_range = (0,1)
# hog_set = [0,1,2,"ALL"]
# for hog_channel in hog_set:
#     t1_f=time.time()
#     car_features = extract_features(cars, cspace=colorspace, spatial_size=(spatial, spatial),
#                             hist_bins=histbin, hist_range=hist_range,orient=orient, 
#                             pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
#                             hog_channel=hog_channel)
#     notcar_features = extract_features(notcars, cspace=colorspace, spatial_size=(spatial, spatial),
#                             hist_bins=histbin, hist_range=hist_range,orient=orient, 
#                             pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
#                             hog_channel=hog_channel)
    
#     t2_f=time.time()
#     print('hog_channel =',hog_channel, 'feature extraction time = ',t2_f-t1_f,'sec')

#     # Create an array stack of feature vectors
#     X = np.vstack((car_features, notcar_features)).astype(np.float64)

#     # Define the labels vector
#     y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))

#     # Split up data into randomized training and test sets
#     rand_state = np.random.randint(0, 100)
#     X_train, X_test, y_train, y_test = train_test_split(
#         X, y, test_size=0.2, random_state=rand_state)

#     # Fit a per-column scaler only on the training data
#     X_scaler = StandardScaler().fit(X_train)
#     # Apply the scaler to X_train and X_test
#     X_train = X_scaler.transform(X_train)
#     X_test = X_scaler.transform(X_test)

#     print('Using spatial binning of:',spatial,
#         'and', histbin,'histogram bins')
    
#     print('Feature vector length:', len(X_train[0]))
#     # Use a linear SVC 
#     svc = LinearSVC()
#     # Check the training time for the SVC
#     t=time.time()
#     svc.fit(X_train, y_train)
#     t2 = time.time()
#     print(round(t2-t, 2), 'Seconds to train SVC...')
#     # Check the score of the SVC
#     print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
#     # Check the prediction time for a single sample
#     t=time.time()
#     n_predict = 10
#     print('My SVC predicts: ', svc.predict(X_test[0:n_predict]))
#     print('For these',n_predict, 'labels: ', y_test[0:n_predict])
#     t2 = time.time()
#     print(round(t2-t, 5), 'Seconds to predict', n_predict,'labels with SVC')
    

hog_channel = 0 feature extraction time =  86.15360069274902 sec
Using spatial binning of: 16 and 32 histogram bins
Feature vector length: 2628
17.87 Seconds to train SVC...
Test Accuracy of SVC =  0.9761
My SVC predicts:  [ 1.  1.  1.  0.  0.  0.  0.  0.  1.  0.]
For these 10 labels:  [ 1.  1.  1.  0.  0.  0.  0.  0.  1.  0.]
0.00216 Seconds to predict 10 labels with SVC
hog_channel = 1 feature extraction time =  58.77270007133484 sec
Using spatial binning of: 16 and 32 histogram bins
Feature vector length: 2628
12.87 Seconds to train SVC...
Test Accuracy of SVC =  0.9772
My SVC predicts:  [ 1.  1.  1.  0.  0.  0.  1.  0.  0.  1.]
For these 10 labels:  [ 1.  1.  1.  0.  0.  0.  1.  0.  0.  1.]
0.0022 Seconds to predict 10 labels with SVC
hog_channel = 2 feature extraction time =  57.4806010723114 sec
Using spatial binning of: 16 and 32 histogram bins
Feature vector length: 2628
13.44 Seconds to train SVC...
Test Accuracy of SVC =  0.9789
My SVC predicts:  [ 1.  0.  1.  0.  1.  0.  0. 

In [ ]:
# Define a function to draw bounding boxes
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    # Make a copy of the image
    imcopy = np.copy(img)
    # Iterate through the bounding boxes
    for bbox in bboxes:
        # Draw a rectangle given bbox coordinates
        cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
    # Return the image copy with boxes drawn
    return imcopy